In [1]:
#writer agent


from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
geminiAPIKey=os.getenv("GEMINI_API_KEY") 

In [ ]:

""" from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client=OpenAIChatCompletionClient(model="gemini-1.5-flash-8b",
api_key=geminiAPIKey) """

In [2]:
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# Model client
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [4]:
from autogen_agentchat.agents import AssistantAgent
writerAgent=AssistantAgent(
    name='writerAgent',
    model_client=model_client,
    description='Summarize the text',
    system_message="you summarize the text provided to you. keep it under 50 words"
)
reviewerAgent = AssistantAgent(
    name = 'reviewerAgent',
    model_client=model_client,
    description='Review the text',
    system_message="You review the summary given by the writer agent and make sure it is understandable based on the context. If you feel that the summary is fine, please say 'APPROVE'"
)

editorAgent = AssistantAgent(
    name = 'editorAgent',
    model_client=model_client,
    description='Edit the text',
    system_message="You edit the summary provided to you to make it easy to understand. keep it under 20 words."
)

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat

from autogen_agentchat.conditions import TextMentionTermination

inner_termination = TextMentionTermination("APPROVE")
writerTeam=RoundRobinGroupChat(participants=[writerAgent,reviewerAgent,editorAgent],termination_condition=inner_termination,max_turns=4)

In [6]:
from autogen_agentchat.agents import SocietyOfMindAgent
summaryAgent = SocietyOfMindAgent("society_of_mind", team=writerTeam, model_client=model_client)


In [7]:
from langchain_community.utilities import GoogleSerperAPIWrapper

SERPER_API_KEY=os.environ['SERPER_API_KEY']

search_tool_wrapper = GoogleSerperAPIWrapper(type='search',serper_api_key=SERPER_API_KEY)

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found." 

In [8]:
search_web("what are 2026 Network Adequacy requirements for GA QHP FFM market for primary care adult?")

"For PY2026, the following four states that perform plan management functions have received CMS approval to conduct their own NA reviews in lieu of CMS ... Missing: GA | Show results with:GA. These instructions are for issuers requesting QHP certification to sell Affordable Care Act (ACA)-compliant plans on Georgia Access for PY 2026, ... ^To count toward meeting network adequacy standards, providers must be appropriately licensed, accredited, or certified to practice in their state, as ... Missing: GA | Show results with:GA. Getting your Qualified Health Plan (QHP) certified for plan year 2026 (PY2026) requires meeting new standards introduced by CMS. This year, it's especially ... Missing: GA FFM primary adult?. Requirements include, but are not limited to: Average travel time from the HMO geographic service area boundary to the nearest primary care ... Missing: 2026 | Show results with:2026. This includes discussions of standardized plan requirements, network adequacy requirements, 

In [17]:
search_web("what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026?")

'The Contract Year (CY) 2026 MA and Part D final rule implements changes related to prescription drug coverage, the Medicare Prescription Payment Plan, dual ... The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type: ... Q9: What is changing for Plan Year 2026 regarding Network Adequacy data submissions, MPMS validations, and CMS review results? CMS is making the following ... This final rule includes a number of new policies that will improve these programs for contract year 2026, as well as codify existing Part C ... The Final Notice of Benefit and Payment Parameters for 2026 is out. Discover the key changes in network adequacy requirements. In the 2026 NBPP, CMS is proposing to change this by requiring issuers to establish a premium payment th

In [10]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web],
    reflect_on_tool_use=True,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do summary of results.
    ''',
)

In [17]:
planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        SummaryAgent: Summarizes the information into simple text

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, if summary agent gave good summary end with "TERMINATE".
    """,
)


In [12]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

finalText_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=10)
combined_termination = finalText_mention_termination | max_message_termination

In [13]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [18]:
from autogen_agentchat.teams import SelectorGroupChat
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, summaryAgent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [ ]:
#task="what are 2026 Network Adequacy requirements for GA QHP FFM market for primary care adult? I only need for Georgia Market for primary care adult specialty. Summarize it in under 50 words"

In [19]:
task="what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 5 changes."

In [20]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 5 changes.
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent: Search for the latest CMS (Centers for Medicare & Medicaid Services) Network Adequacy Requirements for Plan Year 2026 and identify key changes.
2. SummaryAgent: Summarize the identified key changes into the top 5 changes.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_xDyumCZSite5oomvOsJ9PBbd', arguments='{"query":"CMS Network Adequacy requirements changes Plan Year 2026"}', name='search_web')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='The Contract Year (CY) 2026 MA and Part D final rule implements changes related to prescription drug coverage, the Medicare Prescription Payment Plan, dual ... Missing: Adequacy | Show results with:Adequacy. Issuers must only include providers on the

TaskResult(messages=[TextMessage(id='3629cdb0-55cf-4160-a53c-000a7f1ceb27', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 28, 796637, tzinfo=datetime.timezone.utc), content='what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 5 changes.', type='TextMessage'), TextMessage(id='4835fed7-1b53-4ee0-910c-b9c53194c804', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=147, completion_tokens=54), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 30, 793138, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent: Search for the latest CMS (Centers for Medicare & Medicaid Services) Network Adequacy Requirements for Plan Year 2026 and identify key changes.\n2. SummaryAgent: Summarize the identified key changes into the top 5 changes.', type='TextMessage'), ToolCallRequestEvent(id='4dc4e53e-96b8-4762-b6a0-2631cb22c08e', source='WebSearchAgent', models_usage=RequestUsage(pr

TaskResult(messages=[TextMessage(id='3629cdb0-55cf-4160-a53c-000a7f1ceb27', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 28, 796637, tzinfo=datetime.timezone.utc), content='what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 5 changes.', type='TextMessage'), 


TextMessage(id='4835fed7-1b53-4ee0-910c-b9c53194c804', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=147, completion_tokens=54), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 30, 793138, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent: Search for the latest CMS (Centers for Medicare & Medicaid Services) Network Adequacy Requirements for Plan Year 2026 and identify key changes.\n2. SummaryAgent: Summarize the identified key changes into the top 5 changes.', type='TextMessage'), ToolCallRequestEvent(id='4dc4e53e-96b8-4762-b6a0-2631cb22c08e', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=196, completion_tokens=25), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 32, 214866, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_xDyumCZSite5oomvOsJ9PBbd', arguments='{"query":"CMS Network Adequacy requirements changes Plan Year 2026"}', name='search_web')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='a8dd843d-eec6-4aaf-aeeb-f1d130818498', source='WebSearchAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 33, 126101, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Contract Year (CY) 2026 MA and Part D final rule implements changes related to prescription drug coverage, the Medicare Prescription Payment Plan, dual ... Missing: Adequacy | Show results with:Adequacy. Issuers must only include providers on the NA Template that are not at known risk for potential contract termination or non-renewal for the upcoming plan year. Contracts that fail to meet network adequacy requirements during the contract year may be subject to compliance or enforcement actions. Page 8. Medicare ... This final rule includes a number of new policies that will improve these programs for contract year 2026, as well as codify existing Part C ... Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type: ... Q9: What is changing for Plan Year 2026 regarding Network Adequacy data submissions, MPMS validations, and CMS review results? CMS is making the following ... CMS proposes significant reforms to the prior authorization process, requiring plans to adopt electronic systems for streamlining requests, ... Health plans that will submit applications for CY 2026 are required to meet all network adequacy requirements in addition to converting any ... The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. CMS proposes to eliminate the existing requirement at § 422.101(b)(6)(i)(A) that the MAO must demonstrate that an internal coverage criterion ...', name='search_web', call_id='call_xDyumCZSite5oomvOsJ9PBbd', is_error=False)], type='ToolCallExecutionEvent'), 

TextMessage(id='e19b6c86-7b05-45ac-9916-dc82e744d68c', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=501, completion_tokens=50), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 34, 772806, tzinfo=datetime.timezone.utc), content='I found some information related to CMS Network Adequacy requirements for Plan Year 2026, but it did not include a detailed list of key changes. Please confirm if you would like additional assistance or if you have specific sources you want information from.', type='TextMessage'), 

TextMessage(id='2ec21290-dbcc-4f90-bf91-37fa1b4eeaec', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=263, completion_tokens=45), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 40, 359513, tzinfo=datetime.timezone.utc), content='WebSearchAgent: Conduct a more in-depth search for detailed information on the key changes in CMS Network Adequacy Requirements for Plan Year 2026, making sure to check official CMS publications or credible health policy analysis websites.', type='TextMessage'), ToolCallRequestEvent(id='2da4c160-f8ba-474f-a473-0d269358afa8', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=651, completion_tokens=42), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 43, 562903, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_c4qns6jliresFY2Ht2dOmUMJ', arguments='{"query":"key changes CMS Network Adequacy Requirements Plan Year 2026 site:cms.gov OR site:kff.org OR site:healthaffairs.org"}', name='search_web')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='7e5ef567-0cae-4c78-ab33-ba06485a22d9', source='WebSearchAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 44, 325153, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Key Changes for PY2026: NA Template: New columns have been added to the NA ... If the file name is changed, CMS may issue a required correction. In PY2026, issuers will experience changes to the submission of rate filings and in accessing review results and application group and section statuses. Similar to our approach in prior years, CMS will continue to assess for PY2026 whether QHPs in FFEs fulfill the network adequacy time and distance regulatory ... The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. This section describes how CMS will conduct reviews of the network adequacy standards for medical QHP and SADP certification for the 2026 plan ... This year, the change in growth rates from the. Advance Notice to the Rate Announcement is due primarily to incorporation of additional payment ... The Contract Year (CY) 2026 MA and Part D final rule implements changes related to prescription drug coverage, the Medicare Prescription Payment Plan, dual ... Qualified Health Plan. Issuer Application Instructions. Plan Year 2026 ... Key Changes for PY2026. Network Adequacy. ♢ NA ... changed, CMS may issue a required ... Key Changes for PY2026: There is new guidance relating to meaningful difference requirements to ensure compliance with requirements published in the 2026 ... Summary of Key Changes in These Final CY 2026 Program Instructions. CMS received many thoughtful and helpful comments from consumer and patient groups,.', name='search_web', call_id='call_c4qns6jliresFY2Ht2dOmUMJ', is_error=False)], type='ToolCallExecutionEvent'), 

TextMessage(id='b22059e2-a2a6-489c-831d-aec8cfa91ef5', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=963, completion_tokens=235), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 49, 611716, tzinfo=datetime.timezone.utc), content="Here's a summary of key changes in CMS Network Adequacy Requirements for Plan Year 2026, based on search results from official sources:\n\n1. **New Data Submission Requirements**: Updates have been made regarding how data submissions for network adequacy are handled, including adjustments to the submission templates and requirements for files.\n\n2. **Time and Distance Standards Review**: CMS will continue reviewing network adequacy regarding time and distance standards to ensure compliance with updated regulatory requirements for qualified health plans (QHPs).\n\n3. **Rate Filing Changes**: There are changes to how rate filings will be submitted and reviewed, which may impact how issuers access review results and application statuses.\n\n4. **Increased Accountability**: Enhancements are proposed to make MA and Part D plans more accountable for delivering high-quality coverage, although specific accountability measures related to network adequacy weren't detailed.\n\n5. **Meaningful Difference Guidance**: New guidance on meaningful difference requirements has been provided to ensure compliance, potentially impacting how network adequacy is assessed or necessary to meet these standards.\n\nThis summary is based on available information, and for more details, official CMS publications or the final rule should be consulted.", type='TextMessage'), 

TextMessage(id='c041ead8-95fa-421d-bcc5-81ff94985136', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=555, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 50, 855849, tzinfo=datetime.timezone.utc), content='SummaryAgent: Summarize the identified key changes into the top 5 changes based on the information provided by the WebSearchAgent.', type='TextMessage'), 

TextMessage(id='b3a10c3c-ec8f-405c-a72b-597914b7a650', source='writerAgent', models_usage=RequestUsage(prompt_tokens=501, completion_tokens=51), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 53, 414358, tzinfo=datetime.timezone.utc), content='Key changes in CMS Network Adequacy Requirements for Plan Year 2026 include: updates to data submission requirements, ongoing reviews of time and distance standards, changes to rate filing processes, increased accountability for plan quality, and new guidance on meaningful difference requirements.', type='TextMessage'), 

TextMessage(id='668ec720-a77b-491a-a6f8-dc4a3940f4f2', source='reviewerAgent', models_usage=RequestUsage(prompt_tokens=581, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 53, 933036, tzinfo=datetime.timezone.utc), content='APPROVE', type='TextMessage'), 

TextMessage(id='df9db4ce-d0e4-4cb5-8ddd-50d21917c476', source='society_of_mind', models_usage=RequestUsage(prompt_tokens=125, completion_tokens=50), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 54, 999893, tzinfo=datetime.timezone.utc), content='Key changes in CMS Network Adequacy Requirements for Plan Year 2026 include updates to data submission requirements, ongoing reviews of time and distance standards, changes to rate filing processes, increased accountability for plan quality, and new guidance on meaningful difference requirements.', type='TextMessage'), 

TextMessage(id='1240065e-9cd3-47ee-89f2-474b51d4a35d', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=646, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 7, 17, 2, 43, 56, 91477, tzinfo=datetime.timezone.utc), content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")